# Step-By-Step Guide to Creating a Multi-Step Agent using LlamaIndex, CrewAI, Azure OpenAI and Azure AI Search

## Step 1: Install Required Packages

Ensure that you have installed the necessary packages:

In [ ]:
! pip install llama-index
! pip install python-dotenv
! pip install llama-index-readers-huggingface-fs
! pip install llama-index-vector-stores-azureaisearch
! pip install azure-search-documents==11.4.0
! pip install llama-index-embeddings-azure-openai
! pip install llama-index-llms-azure-openai
! pip install datasets
! pip install "crewai[tools]"

In [68]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement

# Environment Variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME") # I'm using GPT-4o
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME") # I'm using text-embedding-3-large
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_SERVICE_API_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
INDEX_NAME = "open-australian-legal-corpus"

# Initialize Azure OpenAI and embedding models
llm = AzureOpenAI(
    model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    deployment_name=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version="2024-02-01"
)

embed_model = AzureOpenAIEmbedding(
    model=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
    deployment_name=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version="2024-02-01"
)

# Initialize search clients
credential = AzureKeyCredential(SEARCH_SERVICE_API_KEY)
index_client = SearchIndexClient(endpoint=SEARCH_SERVICE_ENDPOINT, credential=credential)
search_client = SearchClient(endpoint=SEARCH_SERVICE_ENDPOINT, index_name=INDEX_NAME, credential=credential)


Step 2: Load the Dataset

You can either use the loading methods in Hugging Face `Datasets` directly or use the LlamaIndex `HuggingFaceFSReader`
I'll be doing some data cleanup so I'll use the native pandas loader `read_json` to load my dataset.

In [39]:
from datasets import load_dataset

import pandas as pd

# https://huggingface.co/datasets/umarbutler/open-australian-legal-corpus


# Load only the first 100 rows from the JSONL file
dataset_df = pd.read_json("hf://datasets/umarbutler/open-australian-legal-corpus/corpus.jsonl", lines=True, nrows=100)

In [40]:
dataset_df.head()

,version_id,type,jurisdiction,source,mime,date,citation,url,when_scraped,text
0,south_australian_legislation:46593b41d5646f59/...,secondary_legislation,south_australia,south_australian_legislation,application/rtf,2018-06-14,Fences Regulations 2018 (SA),https://www.legislation.sa.gov.au/__legislatio...,2024-05-25T23:27:15.691552+10:00,South Australia\nFences Regulations 2018\nunde...
1,south_australian_legislation:2021-01-01/1999.4...,primary_legislation,south_australia,south_australian_legislation,application/rtf,2021-01-01,Federal Courts (State Jurisdiction) Act 1999 (SA),https://www.legislation.sa.gov.au/__legislatio...,2024-05-25T23:27:15.730486+10:00,South Australia\nFederal Courts (State Jurisdi...
2,south_australian_legislation:2018-11-01/2005.7...,primary_legislation,south_australia,south_australian_legislation,application/rtf,2018-11-01,Terrorism (Police Powers) Act 2005 (SA),https://www.legislation.sa.gov.au/__legislatio...,2024-05-25T23:27:15.817516+10:00,South Australia\nTerrorism (Police Powers) Act...
3,south_australian_legislation:2023-11-23/2021.5...,primary_legislation,south_australia,south_australian_legislation,application/rtf,2023-11-23,Social Workers Registration Act 2021 (SA),https://www.legislation.sa.gov.au/__legislatio...,2024-05-25T23:27:15.988610+10:00,South Australia\nSocial Workers Registration A...
4,south_australian_legislation:2023-03-30/2021.1...,secondary_legislation,south_australia,south_australian_legislation,application/rtf,2023-03-30,Primary Industry Funding Schemes (Clare Valley...,https://www.legislation.sa.gov.au/__legislatio...,2024-05-25T23:27:16.550214+10:00,South Australia\nPrimary Industry Funding Sche...


I want to see if I need to split the documents prior to vectorization, so I'll check the maximum "text" length.

In [41]:
# Calculate the maximum length of the values in the 'text' column
max_length = dataset_df['text'].apply(len).max()

# Display the maximum length
print(f"The maximum length of the values in the 'text' column is: {max_length}")

The maximum length of the values in the 'text' column is: 560078


The `text-embedding-3-large` embedding model can process a maximum of 8192 tokens as input, which is approximately 32,000 characters of text. Therefore, it is necessary to chunk the text to fit within these limits.

In [42]:
# Drop the 'mime' and 'when_scraped' columns
dataset_df = dataset_df.drop(columns=['mime', 'when_scraped'])

# Verify the columns have been dropped
print(dataset_df.columns)

Index(['version_id', 'type', 'jurisdiction', 'source', 'date', 'citation',
       'url', 'text'],
      dtype='object')


In [43]:
import json
from llama_index.core import Document
from llama_index.core.schema import MetadataMode

# Convert 'date' column to the correct DateTimeOffset string format (ISO 8601 format)
# Assuming that the original date field is in UTC, but if it's not, you might need to adjust the timezone
dataset_df['date'] = dataset_df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

# Ensure other fields are properly cast as strings
dataset_df['type'] = dataset_df['type'].astype(str)
dataset_df['source'] = dataset_df['source'].astype(str)
dataset_df['jurisdiction'] = dataset_df['jurisdiction'].astype(str)
dataset_df['citation'] = dataset_df['citation'].astype(str)
dataset_df['version_id'] = dataset_df['version_id'].astype(str)
dataset_df['url'] = dataset_df['url'].astype(str)
dataset_df['text'] = dataset_df['text'].astype(str)

## Step 3: Convert the DataFrame to LlamaIndex Documents

In [44]:
import json
from llama_index.core import Document
from llama_index.core.schema import MetadataMode

# Convert the DataFrame to a JSON string representation
documents_json = dataset_df.to_json(orient="records")

# Load the JSON string into a Python list of dictionaries
documents_list = json.loads(documents_json)

llama_documents = []

for document in documents_list:
    # Rename 'version_id' to 'parent_id'
    document["parent_id"] = document.pop("version_id")

    # Create a Document object with the text and excluded metadata for llm and embedding models
    llama_document = Document(
        text=document["text"],
        metadata=document,
        excluded_llm_metadata_keys=["text", "url", "parent_id"],
        excluded_embed_metadata_keys=["text", "url", "parent_id", "date"],
        metadata_template="{key}=>{value}",
        text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
    )

    llama_documents.append(llama_document)

# Observing an example of what the LLM and Embedding model receive as input
print(
    "\nThe LLM sees this: \n",
    llama_documents[0].get_content(metadata_mode=MetadataMode.LLM),
)
print(
    "\nThe Embedding model sees this: \n",
    llama_documents[0].get_content(metadata_mode=MetadataMode.EMBED),
)


The LLM sees this: 
 Metadata: type=>secondary_legislation
jurisdiction=>south_australia
source=>south_australian_legislation
date=>2018-06-14T00:00:00.000000Z
citation=>Fences Regulations 2018 (SA)
-----
Content: South Australia
Fences Regulations 2018
under the Fences Act 1975



Contents
1	Short title
2	Commencement
3	Exempt classes of Land
Schedule 1—Revocation of Fences Regulations 2003
Legislative history



1—Short title
These regulations may be cited as the Fences Regulations 2018.
2—Commencement
These regulations come into operation on the day on which they are made.
3—Exempt classes of Land
Pursuant to section 20(3) of the Fences Act 1975, the following classes of land are exempt from the provisions of that Act:
	(a)	land held by the Commissioner of Highways or any council for the purposes of controlling access to a road or proposed road from land abutting the road or proposed road;
	(b)	land of, or used by, the Crown, an instrumentality or agency of the Crown or a council t

## Step 4: Split the Documents 

Note, in this example, I'm including the metadata into my embedding model as an input. In the future, I'd evaluate whether it's necesseary to include this vs only embedding the "text".

Step 2: Create or Use Existing Azure AI Search Index

In [69]:
from llama_index.vector_stores.azureaisearch import (
    IndexManagement,
    MetadataIndexFieldType,
)
import nest_asyncio
from llama_index.core.extractors import TitleExtractor, QuestionsAnsweredExtractor
from llama_index.core.node_parser import TokenTextSplitter

nest_asyncio.apply()


from llama_index.core.settings import Settings

Settings.llm = llm
Settings.embed_model = embed_model

# Metadata fields mapping to Azure AI Search field types
metadata_fields = {
    "source": ("source", MetadataIndexFieldType.STRING),  # String type for source
    "date": (
        "date",
        MetadataIndexFieldType.STRING,
    ),  # String representation of DateTimeOffset
    "citation": ("citation", MetadataIndexFieldType.STRING),  # String type for citation
    "jurisdiction": (
        "jurisdiction",
        MetadataIndexFieldType.STRING,
    ),  # String type for jurisdiction
}

# Set this to True if you want to use an existing index, False to create a new one
use_existing_index = True

# Initialize the vector store
if use_existing_index:
    vector_store = AzureAISearchVectorStore(
        search_or_index_client=index_client,
        index_name=INDEX_NAME,
        index_management=IndexManagement.VALIDATE_INDEX,
        id_field_key="id",
        chunk_field_key="text",
        embedding_field_key="embedding",
        embedding_dimensionality=3072,
        metadata_string_field_key="metadata",
        doc_id_field_key="parent_id",
        filterable_metadata_field_keys=metadata_fields,
        language_analyzer="en.lucene",
        vector_algorithm_type="exhaustiveKnn",
    )
else:
    vector_store = AzureAISearchVectorStore(
        search_or_index_client=index_client,
        index_name=INDEX_NAME,
        index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
        id_field_key="id",
        chunk_field_key="text",
        embedding_field_key="embedding",
        embedding_dimensionality=3072,
        metadata_string_field_key="metadata",
        doc_id_field_key="parent_id",
        filterable_metadata_field_keys=metadata_fields,
        language_analyzer="en.lucene",
        vector_algorithm_type="exhaustiveKnn",
    )

# Load documents and create index based on the use_existing_index flag
if use_existing_index:
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        [],
        storage_context=storage_context,
    )
else:
    # Configure text splitter
    text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

    # Load documents
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Create index
    index = VectorStoreIndex.from_documents(
        llama_documents,
        transformations=[text_splitter],
        storage_context=storage_context,
    )

In [72]:
from llama_index.core.response.notebook_utils import display_response
from llama_index.core.schema import MetadataMode

# Query execution
query = "If a Clare Valley winemaker inadvertently underpays their contributions to the Fund, what are the potential consequences, and is there a grace period for making up the shortfall?"
query_engine = index.as_query_engine(llm, similarity_top_k=3)
response = query_engine.query(query)

# Print the response
display_response(response)
print("\n")

# Print what the LLM sees
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

**`Final Response:`** If a Clare Valley winemaker inadvertently underpays their contributions to the Fund, they may be considered in default if the contributions are not paid within the immediately preceding two prescribed periods. There is no specific grace period mentioned for making up the shortfall, but the winemaker must ensure that all contributions are paid on or before the last day of the month that immediately follows the prescribed period. Failure to do so could result in being in default in relation to contributions to the Fund.



Metadata: type=>secondary_legislation
jurisdiction=>south_australia
source=>south_australian_legislation
date=>2023-03-30T00:00:00.000000Z
citation=>Primary Industry Funding Schemes (Clare Valley Wine Industry Fund) Regulations 2021 (SA)
-----
Content: in respect of a prescribed period after the commencement of the prescribed period, the amount as varied applies only in respect of the part of the prescribed period following the date on which the variation takes effect.
	(7)	A Clare Valley winemaker must—
	(a)	keep proper records relating to the tonnage of Clare Valley grapes delivered to the winemaker, the growers of those grapes and the contributions required to be made (on the winemaker's own behalf and on behalf of growers) in respect of those grapes; and
	(b)	make those records available for inspection at any reasonable time by a person authorised by the Minister for the purpose.
	(8)	A Clare Valley winemaker must, on or before the last day of the month that immediately follows a

# Explanation of the Scenario and Process

## Scenario
The scenario revolves around a real-world issue where a winemaker has underpaid their contributions to a regulatory fund. The winemaker needs to know the legal consequences and any steps they can take to correct the issue.

## Multi-Step Reasoning

- **Legal Document Reviewer:** Reviews relevant regulations to identify key clauses that impact the winemaker's situation.
- **Compliance Expert:** Assesses the compliance risks and provides recommendations for how to rectify the issue.
- **Case Law Researcher:** Looks for relevant case law that can guide the legal strategy.
- **Legal Drafter:** Drafts a formal legal response to the regulatory body on behalf of the winemaker.

## Why This Matters

- **Practical Use:** This multi-step process showcases how a legal team might collaboratively handle a real-world legal issue using AI tools.
- **Clear Roles and Outputs:** Each agent has a clear role and expected output, which mirrors how legal professionals operate in practice.
- **AI Augmentation:** By leveraging AI for document retrieval, compliance checks, and drafting, this process demonstrates the practical applications of AI in legal workflows.

This setup can be a strong showcase of how AI can assist in legal scenarios, providing value both in automating mundane tasks and enhancing decision-making through retrieval-augmented intelligence.

In [74]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import LlamaIndexTool
from langchain_openai import AzureChatOpenAI

# Define Azure OpenAI configuration
azure_llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
)

# Define query engine from your LlamaIndex setup
query_engine = index.as_query_engine(similarity_top_k=5)

# Query tool for all agents
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Legal Document Query Tool",
    description="Use this tool to look up legal documents, case law, and compliance information.",
)

# Agents
legal_review_agent = Agent(
    role="Senior Legal Document Reviewer",
    goal="Review and summarize legal documents, flag important clauses, and retrieve relevant case law.",
    backstory=(
        "You are an experienced legal professional with expertise in Australian legislation "
        "and case law. You have been tasked with reviewing the legal obligations of a Clare Valley "
        "winemaker who has underpaid their contributions to the Fund."
    ),
    verbose=True,
    tools=[query_tool],
    max_execution_time=60,
    max_rpm=None,
    memory=True,  # Enable memory,
    llm=azure_llm,
)

compliance_agent = Agent(
    role="Legal Compliance Expert",
    goal="Identify and flag compliance-related issues, ensuring adherence to legal standards.",
    backstory=(
        "You specialize in legal compliance, with a focus on ensuring that organizations "
        "meet their regulatory obligations. Your task is to assess the compliance risks "
        "associated with the winemaker's underpayment and determine how to rectify the issue."
    ),
    verbose=True,
    tools=[query_tool],
    max_execution_time=60,
    max_rpm=None,
    memory=True,  # Enable memory
    llm=azure_llm,
)

case_law_researcher = Agent(
    role="Case Law Researcher",
    goal="Retrieve relevant case law and provide context for legal decisions.",
    backstory=(
        "You are a legal researcher focused on finding and analyzing relevant legal precedents. "
        "In this case, you need to identify any case law that provides guidance on similar compliance "
        "issues within the wine industry or related sectors."
    ),
    verbose=True,
    tools=[query_tool],
    max_execution_time=60,
    max_rpm=None,
    memory=True,  # Enable memory
    llm=azure_llm,
)

legal_drafter = Agent(
    role="Legal Drafting Specialist",
    goal="Draft legal documents that are compliant with legal standards and aligned with organizational goals.",
    backstory=(
        "You are a skilled legal drafter responsible for creating precise and legally sound documents. "
        "In this task, you will draft a formal response on behalf of the winemaker to the regulatory body, "
        "addressing the underpayment issue and proposing a compliant solution."
    ),
    verbose=True,
    max_execution_time=60,
    max_rpm=None,
    memory=True,  # Enable memory
    llm=azure_llm,
)

# Tasks
review_task = Task(
    description=(
        "Review the Clare Valley Wine Industry Fund regulations and summarize the key points. "
        "Flag any important clauses or terms that require further attention, particularly those "
        "related to underpayment and compliance."
    ),
    expected_output="Summarized report with flagged clauses.",
    agent=legal_review_agent,
)

compliance_task = Task(
    description=(
        "Analyze the summary and identify any compliance issues or potential risks associated with "
        "the winemaker's underpayment. Provide recommendations on how to rectify the situation."
    ),
    expected_output="Compliance report with risk assessment and recommendations.",
    agent=compliance_agent,
)

case_law_task = Task(
    description=(
        "Retrieve relevant case law that may provide guidance on similar compliance issues in the wine industry. "
        "Analyze how these precedents might impact the current legal situation."
    ),
    expected_output="Case law report with analysis.",
    agent=case_law_researcher,
)

drafting_task = Task(
    description=(
        "Draft a formal legal response on behalf of the winemaker to the regulatory body. "
        "The document should address the underpayment issue, propose a compliant solution, "
        "and reference relevant regulations and case law."
    ),
    expected_output="Drafted legal document in markdown format of max 2000 characters.",
    agent=legal_drafter,
)

# Crew process
crew = Crew(
    agents=[legal_review_agent, compliance_agent, case_law_researcher, legal_drafter],
    tasks=[review_task, compliance_task, case_law_task, drafting_task],
    verbose=True,
    process=Process.sequential,
)

# Kick off the process
result = crew.kickoff()
print(result)


2024-08-21 11:41:28,263 - 34532 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-21 11:41:28][DEBUG]: == Working Agent: Senior Legal Document Reviewer
 [2024-08-21 11:41:28][INFO]: == Starting Task: Review the Clare Valley Wine Industry Fund regulations and summarize the key points. Flag any important clauses or terms that require further attention, particularly those related to underpayment and compliance.


> Entering new CrewAgentExecutor chain...
Thought: I need to gather information about the Clare Valley Wine Industry Fund regulations, particularly focusing on clauses related to underpayment and compliance.

Action: Legal Document Query Tool
Action Input: {"query": "Clare Valley Wine Industry Fund regulations underpayment compliance"} 

Under the Clare Valley Wine Industry Fund regulations, compliance regarding underpayment involves several key points. Clare Valley winemakers are required to keep proper records of the tonnage of Clare Valley grapes delivered, the growers of those grapes, and the contributions made on behalf of both the winemakers an

Once you run the code, your agent will review the legal document and provide a summarized report. It will flag important clauses and retrieve related case law as needed. Here's an example output you might expect:

# Breakdown of the CrewAI Multi-Step Legal Process

This markdown provides a detailed analysis and documentation of the CrewAI process that simulates a real-world legal scenario involving a Clare Valley winemaker. The scenario revolves around the winemaker's underpayment issue and the steps taken to address the compliance and legal implications.

## Step 1: Senior Legal Document Reviewer's Task

**Objective:** Review the Clare Valley Wine Industry Fund regulations, summarize key points, and flag any important clauses related to underpayment and compliance.

### Summary of Findings:

- **Record-Keeping Requirements:** Winemakers must maintain records of grape tonnage and contributions, which must be available for inspection.
- **Financial Statements and Contributions:** Winemakers must submit financial statements and contributions by the end of the month following each prescribed period.
- **Penalties for Non-Compliance:** Non-compliance may result in disqualification from benefits and penalties of up to $5,000.

### Important Clauses:

- **Inspection of Records:** Transparency in record-keeping is essential.
- **Submission Deadline:** Timely submission of financial statements is crucial.
- **Penalties for False Information:** Severe penalties underscore the importance of honesty in compliance.

## Step 2: Legal Compliance Expert's Task

**Objective:** Analyze the document review summary and identify compliance issues, providing recommendations for rectifying the underpayment.

### Risk Assessment:

- **Record-Keeping Non-Compliance:** Risks related to inadequate records may result in penalties.
- **Financial Statement Submission Deadline:** Late submissions may lead to penalties and disqualification.
- **Penalties for Non-Compliance:** Non-compliance can result in fines and loss of benefits.

### Recommendations for Rectification:

- **Review and Update Records:** Ensure accurate record-keeping.
- **Timely Submission:** Adhere to submission deadlines with a compliance calendar.
- **Rectify Underpayments:** Proactively correct any shortfalls.
- **Internal Compliance Audits:** Regular audits will ensure ongoing compliance.
- **Training and Awareness:** Educate personnel on compliance requirements.

## Step 3: Case Law Researcher's Task

**Objective:** Retrieve relevant case law and analyze how it impacts the current legal situation for the winemaker.

### Relevant Case Law:

- **Deputy Commissioner of Taxation v Patonga Beach Holdings Pty Limited:** Emphasizes the importance of accurate financial statements and compliance with obligations.
- **Australian Securities and Investments Commission v Healey (2011):** Directors’ responsibility in ensuring accuracy in financial reporting.

### Recommendations:

- **Accurate Record-Keeping:** Essential for demonstrating compliance.
- **Timely Submission of Financial Statements:** Prevent penalties and disqualification.
- **Rectify Underpayments:** Correct underpayments promptly.
- **Internal Compliance Audits:** Regular audits help prevent compliance issues.
- **Training and Awareness:** Training programs reinforce compliance with regulations.

## Step 4: Legal Drafting Specialist's Task

**Objective:** Draft a formal legal response to the regulatory body, addressing the underpayment issue, proposing a compliant solution, and referencing relevant regulations and case law.

### Drafted Legal Response:

[Winemaker's Letterhead]

[Date]

[Regulatory Body's Name]  
[Address]  
[City, State, ZIP Code]

**Subject**: Response to Underpayment Issue and Proposed Compliant Solution

Dear [Regulatory Body's Name],

We acknowledge the underpayment issue in our payroll practices and are committed to rectifying the situation promptly.

### Proposed Compliant Solution
1. **Immediate Compensation Adjustment**: Review and compensate for underpayments.
2. **Implementation of Fair Employment Practices**: Ensure contracts are compliant and fair.
3. **Regular Payroll Audits**: Conduct audits to ensure compliance.
4. **Training and Awareness Programs**: Educate staff on compliance requirements.

### Reference to Relevant Case Law
The case **Crowe v Ho [1996] IRCA 543** emphasizes the importance of procedural and substantive fairness in employment practices. This case serves as a precedent for rectifying underpayment issues.

We are dedicated to ensuring compliance and appreciate your cooperation.

Yours sincerely,  
[Your Name]  
[Your Position]  
[Winemaker's Company Name]  
[Contact Information]



# Comparison of Scenario 1 (Single RAG Chat) and Scenario 2 (CrewAI Multi-Step Process)

## Scenario 1: Single RAG Chat

In this scenario, a single Retrieval-Augmented Generation (RAG) model is used to answer a specific query regarding the consequences of underpayment by a Clare Valley winemaker. The process involves querying the LLM using the legal documents and retrieving relevant information to provide a concise answer.

### Output:

**Final Response:** If a Clare Valley winemaker inadvertently underpays their contributions to the Fund, they may be considered in default if the contributions are not paid within the immediately preceding two prescribed periods. There is no specific grace period mentioned for making up the shortfall, but the winemaker must ensure that all contributions are paid on or before the last day of the month that immediately follows the prescribed period. Failure to do so could result in being in default in relation to contributions to the Fund.

### Key Characteristics:

- **Speed and Efficiency:** The RAG model provides a direct and concise answer quickly, focusing on the specific query without requiring additional steps.
- **Focus:** The response is narrowly tailored to the query and does not involve a broader analysis or exploration of related legal concepts or compliance issues.
- **Simplicity:** The single-chat output is straightforward and easy to understand, making it ideal for quick answers to specific legal questions.
- **Limitations:** The RAG response lacks depth, analysis, and recommendations that could be critical for more complex legal scenarios. It does not explore potential risks, compliance strategies, or provide actionable steps for the winemaker to address the issue.



## Scenario 2: CrewAI Multi-Step Process

In this scenario, a multi-agent CrewAI process is set up with different agents, each specializing in a specific aspect of the legal task. The process involves reviewing the legal document, analyzing compliance issues, retrieving relevant case law, and drafting a formal legal response.

### Output:

- **Detailed Summary and Flagging of Key Clauses:** The Senior Legal Document Reviewer provides a detailed summary of the Clare Valley Wine Industry Fund regulations, highlighting record-keeping requirements, financial statement submissions, and penalties for non-compliance.
- **Comprehensive Compliance Report:** The Legal Compliance Expert identifies compliance risks and provides actionable recommendations, such as reviewing records, timely submission of financial statements, and conducting internal audits.
- **Case Law Analysis:** The Case Law Researcher retrieves and analyzes relevant case law, providing context for how legal precedents can impact the current situation.
- **Formal Legal Response Draft:** The Legal Drafting Specialist drafts a formal letter addressing the underpayment issue, proposing a compliant solution, and referencing relevant case law.

### Key Characteristics:

- **Depth and Thoroughness:** The multi-step process goes beyond answering the immediate query. It provides a comprehensive analysis of the legal and compliance landscape, considering various aspects of the situation.
- **Collaborative Problem-Solving:** By involving multiple agents with specialized roles, the process simulates a real-world legal team working together to address a complex issue. Each agent contributes unique expertise, resulting in a more holistic solution.
- **Actionable Recommendations:** Unlike the RAG model, the CrewAI process offers practical steps for the winemaker to rectify the underpayment, including conducting audits, updating records, and training personnel.
- **Legal Drafting:** The process concludes with a formal legal response, ensuring that the winemaker is equipped with a well-structured communication to the regulatory body, backed by legal precedents and compliance strategies.
- **Complexity:** While the process is more detailed and thorough, it is also more complex and time-consuming compared to a single RAG chat. This level of detail is more suitable for complex legal scenarios where multiple factors must be considered.

# Comparison and Contrast

## Purpose and Scope

- **Scenario 1:** The single RAG chat is best suited for quick, specific queries where a concise answer is sufficient. It focuses on providing an immediate response without delving into broader implications or offering recommendations.
- **Scenario 2:** The CrewAI multi-step process is designed for complex legal scenarios where a comprehensive analysis is required. It addresses not just the immediate query but also the broader context, including compliance, legal risks, and strategic recommendations.

## Depth of Analysis

- **Scenario 1:** Offers a focused but limited response, providing just the essential information based on the query.
- **Scenario 2:** Delivers an in-depth analysis that includes detailed legal summaries, compliance risk assessments, case law research, and a formal legal response, ensuring all aspects of the situation are covered.

## Actionable Output

- **Scenario 1:** Provides a final answer that informs the winemaker of the consequences of underpayment, but lacks actionable steps or broader recommendations.
- **Scenario 2:** Goes further by offering specific, actionable recommendations, such as rectifying underpayments, conducting audits, and training staff, all of which are crucial for legal compliance and risk management.

## Complexity and Time

- **Scenario 1:** Faster and simpler, ideal for quick queries.
- **Scenario 2:** More complex and time-consuming, but necessary for situations that require a comprehensive legal strategy.

## Realism and Practicality

- **Scenario 1:** Simulates a basic chatbot interaction, useful for quick legal inquiries.
- **Scenario 2:** Mimics a real-world legal process, where multiple experts collaborate to provide a robust and legally sound solution.

## Conclusion

- **Scenario 1 (Single RAG Chat):** Ideal for quick, specific queries where a concise answer suffices, but it may fall short when the situation requires a more detailed analysis or strategic recommendations.
- **Scenario 2 (CrewAI Multi-Step Process):** Better suited for complex legal scenarios that require collaboration between multiple experts, offering a thorough and actionable solution that covers all relevant aspects of the issue.

In a real-world legal setting, **Scenario 2** provides greater value due to its depth, thoroughness, and ability to generate actionable recommendations and formal legal responses. However, **Scenario 1** remains valuable for efficiency and simplicity in less complex situations.